## Imports

In [1]:
# !conda install -c conda-forge category_encoders -y
# !conda install -c anaconda scikit-learn -y
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)

import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

In [2]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Bill'

connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

sql_query = "Select * from claims_data"



In [3]:
df = pd.read_sql_query(sql_query, connection)

In [4]:
display(df.describe())
print(df.dtypes)
df

,age,sex,bmi,steps,children,smoker,region,insurance_claim,claim_amount
count,1338,1338,1338,1338,1338,1338,1338,1338,1338
unique,47,2,548,55,6,2,4,2,783
top,18,male,32.3,8008,0,no,southeast,yes,0
freq,69,676,13,52,574,1064,364,783,555


age                object
sex                object
bmi                object
steps              object
children           object
smoker             object
region             object
insurance_claim    object
claim_amount       object
dtype: object


,age,sex,bmi,steps,children,smoker,region,insurance_claim,claim_amount
0,19,female,27.9,3009,0,yes,southwest,yes,16884.924
1,18,male,33.77,3008,1,no,southeast,yes,1725.5523
2,28,male,33,3009,3,no,southeast,no,0
3,33,male,22.705,10009,0,no,northwest,no,0
4,32,male,28.88,8010,0,no,northwest,yes,3866.8552
...,...,...,...,...,...,...,...,...,...
1333,50,male,30.97,4008,3,no,northwest,no,0
1334,18,female,31.92,3003,0,no,northeast,yes,2205.9808
1335,18,female,36.85,3008,0,no,southeast,yes,1629.8335
1336,21,female,25.8,8009,0,no,southwest,no,0


In [5]:
# Select all attribute columns
float_cols = ['age', 'bmi', 'steps', 'children', 'claim_amount']

def to_float(x):    
    "Transforms attribute columns to type float"
    
    if type(x) is int:
        return float(x)
    else:
        return float(x[0:2])

df[float_cols] = df[float_cols].applymap(to_float)

# cat_cols = ['sex', 'smoker', 'region', 'insurance_claim']

# def to_cat(x):    
#     "Transforms attribute columns to type category"
#     scaler = StandardScaler()
#     Xy = scaler.fit_transform(x)
#         return float(x)
#     else:
#         return float(x[0:2])
# df[cat_cols] = df[cat_cols].applymap(to_cat)


## Encode cateogrical data

In [6]:
# df_['insurance_claim'] = df_['insurance_claim'].replace({-0.841910: 0, 1.187776:1})
obj_cols = df.dtypes[df.dtypes == object].index.values
# instantiate an encoder - here we use ordinal (no particular order since we don't specify mapping)
ce_ord = ce.OrdinalEncoder(cols = obj_cols)
df_ = df
df_ = ce_ord.fit_transform(df_)

## Scale data to unit variance

In [7]:
scaler = StandardScaler()
scale_cols = ['age', 'sex', 'bmi', 'steps', 'children', 'smoker', 'region']
df_[scale_cols] = scaler.fit_transform(df_[scale_cols])

## Train/Test Split

In [8]:
from sklearn.model_selection import train_test_split as tts
X = df_[scale_cols[:-1]].fillna(0)
y = df_['claim_amount']

X_train, X_test, y_train, y_test = tts(X, y, random_state = 47, test_size = .2)

## KNN

In [9]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
predicted = knn.predict(X_test)


In [10]:
from sklearn.metrics import mean_squared_error
print('RMSE:', mean_squared_error(y_test, predicted) ** 0.5)

RMSE: 15.432718683697608


## Create Data Frame

In [11]:
predicted = np.array(predicted)
y_test = np.array(y_test)
df = pd.DataFrame(data = [y_test, predicted]).T
df.columns = ['actuals', 'predictions']

## Print Results

In [12]:
df

,actuals,predictions
0,48.0,47.6
1,0.0,0.0
2,76.0,79.0
3,21.0,19.4
4,57.0,71.6
...,...,...
263,0.0,0.0
264,0.0,23.4
265,45.0,13.2
266,40.0,42.4


# DECISION TREE

In [13]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=47)
tree.fit(X_train, y_train)
predicted = tree.predict(X_test)

In [14]:
from sklearn.metrics import mean_squared_error
print('RMSE:', mean_squared_error(y_test, predicted) ** 0.5)

RMSE: 10.677471401150559


## Create Data Frame

In [15]:
predicted = np.array(predicted)
y_test = np.array(y_test)
df = pd.DataFrame(data = [y_test, predicted]).T
df.columns = ['actuals', 'predictions']

## Print Results

In [16]:
df

,actuals,predictions
0,48.0,47.0
1,0.0,0.0
2,76.0,80.0
3,21.0,22.0
4,57.0,54.0
...,...,...
263,0.0,0.0
264,0.0,0.0
265,45.0,39.0
266,40.0,41.0


# RANDOM FOREST

In [17]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=100, random_state=47)
forest.fit(X_train, y_train)
predicted= forest.predict(X_test)

In [18]:
from sklearn.metrics import mean_squared_error
print('RMSE:', mean_squared_error(y_test, predicted) ** 0.5)

RMSE: 10.48193494678333


## Create Data Frame

In [19]:
predicted = np.array(predicted)
y_test = np.array(y_test)
df = pd.DataFrame(data = [y_test, predicted]).T
df.columns = ['actuals', 'predictions']

## Print Results

In [20]:
df

,actuals,predictions
0,48.0,47.600000
1,0.0,0.000000
2,76.0,75.370000
3,21.0,25.172714
4,57.0,45.445000
...,...,...
263,0.0,0.000000
264,0.0,0.000000
265,45.0,25.830000
266,40.0,42.990000


## Send to Database

In [21]:
df.to_sql('regression_model_results___', con = connection)

# SUPPORT VECTOR MACHINE

In [22]:
from sklearn.svm import SVR
svm = SVR(kernel='linear', C=500, gamma=50)
svm.fit(X_train, y_train)
predicted= svm.predict(X_test)

In [23]:
from sklearn.metrics import mean_squared_error
print('RMSE:', mean_squared_error(y_test, predicted) ** 0.5)

RMSE: 22.66251706381464


## Create Data Frame

In [24]:
predicted = np.array(predicted)
y_test = np.array(y_test)
df = pd.DataFrame(data = [y_test, predicted]).T
df.columns = ['actuals', 'predictions']

## Print Results

In [25]:
df

,actuals,predictions
0,48.0,42.250559
1,0.0,-5.441245
2,76.0,10.169859
3,21.0,10.792182
4,57.0,24.317568
...,...,...
263,0.0,0.698095
264,0.0,6.589405
265,45.0,20.901388
266,40.0,36.720069
